# Script to analyse and preprocess the data from the WikiSQL dataset.
-------------------------------------------------------------------------------------------------------------------
# Technology used: basic python preprocessing packages and json parsers

I start with the usual cells for utility purposes.

In [2]:
# packages used for processing: 
import cPickle as pickle # for reading the data
import matplotlib.pyplot as plt # for visualization
import numpy as np

# for json parsing:
import json

# for operating system related stuff
import os
import sys # for memory usage of objects
from subprocess import check_output

# to plot the images inline
%matplotlib inline

In [3]:
# Input data files are available in the "../Data/" directory.

def exec_command(cmd):
    '''
        function to execute a shell command and see it's 
        output in the python console
        @params
        cmd = the command to be executed along with the arguments
              ex: ['ls', '../input']
    '''
    print(check_output(cmd).decode("utf8"))

In [4]:
# check the structure of the project directory
exec_command(['ls', '..'])

Data
Models
Scripts



In [387]:
''' Set the constants for the script '''

# various paths of the files
data_path = "../Data/WikiSQL/data" # the data path

train_files = {
    "questions": os.path.join(data_path, "train.jsonl"),
    "tables": os.path.join(data_path, "train.tables.jsonl")
}

base_model_path = '../Models'

processed_data_file_path = os.path.join(data_path, "processed.pickle")

In [6]:
# check the contents of the data path
exec_command(['ls', data_path])

dev.db
dev.jsonl
dev.tables.jsonl
test.db
test.jsonl
test.tables.jsonl
train.db
train.jsonl
train.tables.jsonl



## The relevant files are in jsonl format. Let's load the train.jsonl file and check out its contents

In [40]:
# function to generate a list of json objects as python dictionaries.
def preliminary_parsing(files): 
    '''
        function to extract the preliminary data from the json files and represent them in 
        python dictionary format
        @param
        files: a dict object that has the questions and tables files
    '''
    
    # extract the two files from the input parameter
    (questions, tables) = (files["questions"], files["tables"])
    
    # first generate the questions data:
    qdata = list() # initialize to empty list
    with open(questions, 'r') as ques:
        for line in ques:
            qdata.append(json.loads(line))
            
    # now generate the tables data
    # the code for doing this quite same as before
    tabdata = list() # initialize to empty list
    with open(tables, 'r') as tabs:
        for line in tabs:
            tabdata.append(json.loads(line))

    # transform the tabdata into a format easy for processing
    modified_tabdata = {}
    for item in tabdata:
        key = item['id']
        del item['id'] # delete the id from the table object
        modified_tabdata[key] = item
            
    # return the extracted data:
    return qdata, modified_tabdata

In [46]:
qdata, tabdata = preliminary_parsing(train_files)
len(qdata) # print the no. of examples to train on

61297

In [47]:
# print a sample example to train on.
qdata[np.random.randint(len(qdata))]

{u'phase': 2,
 u'question': u'How many bronzes were won for the country that had a larger than 3 rank and a silver win count above 0?',
 u'sql': {u'agg': 4, u'conds': [[0, 1, 3], [3, 1, 0]], u'sel': 4},
 u'table_id': u'2-1767441-2'}

## Run the above cell a few number of times to get a feeling of what the data is

In [378]:
# function to create a query from the given python dictionary
def generate_query(que, table_data):
    '''
        This returns a query string generated from the python dictionary
        @param
        que => the query in python dictionary format
        table_data => the data of the tables (from the dataset itself)
        
        @return
        Query string for the given query
    '''
    
    # extract the building parameters of the query
    params = que["sql"]
    
    # The aggregation and condition ops in the format taken from the query dependency in lib:
    agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
    cond_ops = ['=', '>', '<', 'OP']
    
    # initialize query
    query = ""
    
    # generate select part of the query:
    agg_operation = params["agg"]
    table_columns = table_data[que["table_id"]]["header"]
    select_column = table_columns[params["sel"]]
    
    if(agg_operation != 0):
        query += "SELECT " + agg_ops[agg_operation] + ' ( ' + select_column + ' ) '
    else:
        query += "SELECT " + select_column
    
    # keep the table name generic since they will only add randomness to the learning
    query += " FROM <TABLE>"
    
    # add the where and conditions clause so on:
    conditions = []
    for cond in params['conds']:
        col_index = cond[0]; cond_op = cond[1]; cond_target = cond[2]
        
        # add a condition
        conditions.append(table_columns[col_index] + " " 
                          + cond_ops[cond_op] + " " + unicode(cond_target))
    if(len(conditions) != 0):
        # conditions not empty:
        condition_clause = reduce(lambda x,y: x + " AND " + y, conditions)

        query += " WHERE " + condition_clause
        
    # return the query so formed
    return query
    

In [379]:
random_query = qdata[np.random.randint(len(qdata))]
print "Original format: " + str(random_query) + "\n\n"
print "Generated query: " + generate_query(random_query, tabdata)

Original format: {u'phase': 1, u'table_id': u'1-10128185-2', u'question': u'How many votes did Northern Ireland cast if the total was 35?', u'sql': {u'agg': 0, u'sel': 2, u'conds': [[7, 0, 35]]}}


Generated query: SELECT Northern Ireland FROM <TABLE> WHERE Total = 35


In [380]:
# generate the lists of input and ideal_output from the given dataset by using the so defined generate_query 
# function

questions = np.array([elem["question"] for elem in qdata])
queries = np.array([generate_query(elem, tabdata) for elem in qdata])

# Basic preprocessing of the data has been completed. Run the below cell a few times to make sure that the data has been properly processed

In [401]:
# now: check the ideal output for a random question from the dataset
random_index = np.random.randint(questions.shape[0])

print "Random sample from the dataset:\n\n"

# print the natural language question:
print "Natural_Language question: " + questions[random_index]
print "SQL query for the same   : " + queries[random_index]

Random sample from the dataset:


Natural_Language question: Who was the top goalscorer for season 2001-02?
SQL query for the same   : SELECT Top Goalscorer FROM <TABLE> WHERE Season = 2001-02


# Now pickle this processed data to be used later on

In [386]:
processed_data = {
    "questions": questions,
    "queries": queries
}

In [388]:
with open(processed_data_file_path, "w") as dumping:
    pickle.dump(processed_data, dumping, protocol=pickle.HIGHEST_PROTOCOL)

# The data has been processed, now this can be used further to generate embeddings and later on for training the network